Import the necessary packages.

In [1]:
import nltk
import json
import unicodecsv as csv
import numpy as np
import math
import re
import operator
from collections import defaultdict
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from scipy.spatial.distance import cosine as cos_distance
from sklearn.feature_extraction import DictVectorizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import wordnet
from nltk import pos_tag
import string
from sklearn.metrics import f1_score
import tensorflow as tf
from nltk.tag.stanford import StanfordNERTagger
from sklearn.model_selection import train_test_split
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

Init the lists for storing trianing data, test data, and documents.

In [2]:
training_set = []  # list of dict with questions, answers, answer_para, docid
test_set = []  # list of sorted dict with questions, docid, id (question id, not using)
devel_set = []  # list of dict with questions, answers, answer_para, docid
docs = []  # list of sorted dict with docid and text (list of paras)

Read all the data from the .json files and store in the corresponding lists。

In [3]:
# Read documents data
with open('documents.json') as doc_data:
    docs = json.load(doc_data)  # length = 441
doc_data.close()

# Read training data
with open('training.json') as training_data:
    training_set = json.load(training_data)  # length = 43379
training_data.close()

# Read test data
with open('testing.json') as test_data:
    test_set = json.load(test_data)  # length = 3618
test_data.close()

# Read develop data
with open('devel.json') as devel_data:
    devel_set = json.load(devel_data)  # length = 3618
devel_data.close()

## Section 2: Preprocessing

Define a function to tokenize, convert to lowercase, remove the stop words and punctuations, lemmatize the words, and finally get the BOW for the given text (a string)

In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = wordnet.WordNetLemmatizer()

def get_BOW(text):
    word_ls = word_tokenize(text)  # tokenize
    new_ls = []
    for word in word_ls:
        word = word.lower()  # to lower case
        if ((word not in stop_words) and not (re.match("[^\w\s]", word))):  # remove stop words and punctuations
            lemma1 = lemmatizer.lemmatize(word,"v")
            lemma2 = lemmatizer.lemmatize(word,"n")
            if (len(lemma1) < len(lemma2)):
                new_ls.append(lemma1)
            else:
                new_ls.append(lemma2)
        
    # get BOW
    BOW = {}
    for word in new_ls:
        BOW[word] = BOW.get(word,0) + 1
    return BOW

Define a function to compute the inverted index the document. The inverted index find the (para_id, weight) pairs for each term, where weight = normalised tf-idf.

In [5]:
def get_inverted_index(text):
    # tf
    tf_ls = []
    for para in text: # get BOW for each paragraph
        para_tf = get_BOW(para)
        tf_ls.append(para_tf)
    
    # df
    df = {}  # paragraph frequency of the terms
    for para_tf in tf_ls:
        for term in para_tf.keys():
            df[term] = df.get(term,0) + 1
            
    M = len(tf_ls)  # total number of paragraphs in the doc
    
    # inverted index
    inverted_index = defaultdict(list)  # term -> (docid,weight) pairs
    
    # tf-idf values
    for para_id in range(len(tf_ls)):
        para_tf = tf_ls[para_id]
        N = sum(para_tf.values())  # row sum
        vec_len = 0  # vector length
        
        tfidf = {}
        for (term,count) in para_tf.items():
            tfidf_value = float(count) / N * math.log(M / float(df[term]))
            tfidf[term] = tfidf_value
            vec_len += pow(tfidf_value,2)  # squared sum for euclidean distance
        
        # normalise paragraph by vector length and insert into index
        vec_len = pow(vec_len,0.5)   # sqrt for euclidean distance
        for (term, tfidf_value) in tfidf.items():
            if vec_len == 0:
                weight = 0
            else:
                weight = tfidf_value/vec_len
            inverted_index[term].append([para_id,weight])  # weight = normalised tf-idf
    return inverted_index

Get the inverted index for each document, and store as a new attribute.

In [6]:
for doc in docs:
    doc["inverted_index"] = get_inverted_index(doc["text"])

 Define a function to solve for the para_id with highest score in the document for each the question (query).
 If no paragraph contains any key words of the query, choose the first paragraph as the best.

In [7]:
def query_vsm(query, inverted_index):
    accumulator = {}
    for term in query:
        postings = inverted_index[term]
        for (para_id, weight) in postings:
            accumulator[para_id] = accumulator.get(para_id,0) + weight
    if (len(accumulator) == 0):  # no paragraph contains query terms, just choose first paragraph
        return 0
    return max(accumulator.items(), key=operator.itemgetter(1))[0]  # para with max score

Define a function to find the best paragraph for each question, and store as a new attribute for each question in the question set.

In [8]:
def find_best_paras(question_set):
    for info in question_set:
        # Get query terms
        query_terms = get_BOW(info["question"]).keys()
        
        # Get inverted_index for the document
        inverted_index = docs[info["docid"]]["inverted_index"]
        
        # Find the index of the best paragraph
        info["best_para_id"] = query_vsm(query_terms, inverted_index)

Define a function to evaluate the prediciton of answer paragraph.

In [9]:
def evaluate_best_paras(question_set):
    correct = 0
    for info in question_set:
        prediction = info["best_para_id"]
        golden_para = info["answer_paragraph"]
        if (prediction == golden_para):
            correct += 1
    return correct*1.0/len(question_set)

Test the paragraph accuracy with the development set.

In [10]:
# Get best para_id for each question in the development set
find_best_paras(devel_set)

# Evaluate the prediction
accuracy = evaluate_best_paras(devel_set)

print("accuracy: %f" % (accuracy))

accuracy: 0.736842


## Section 3: Common Methods for Basic and Advanced methods

Write a function to evaluate with average F-score. (Shared by both basic and advanced method)

In [11]:
def evaluate_answer(question_set, answers):
    total_score = 0
    for i in range(len(question_set)):
        prediction_tokens = word_tokenize(answers[i])
        golden_standard_tokens = word_tokenize(question_set[i]["text"])
        
        # precision
        precision = 0
        for word in prediction_tokens:
            if word in golden_standard_tokens:
                precision += 1
        if (len(prediction_tokens) == 0):
            precision = 0
        else:
            precision /= len(prediction_tokens)
        
        # recall
        recall = 0
        for word in golden_standard_tokens:
            if word in prediction_tokens:
                recall += 1
        recall /= len(golden_standard_tokens)
        
        if (recall + precision == 0):
            fscore = 0
        else:
            fscore = 2*(precision*recall)/(precision + recall)
        
        total_score += fscore
    return (total_score/len(question_set))

def evaluate_answer_recall(question_set, answers):
    total_recall = 0
    for i in range(len(question_set)):
        prediction_tokens = word_tokenize(answers[i])
        golden_standard_tokens = word_tokenize(question_set[i]["text"])
        
        # recall
        recall = 0
        for word in golden_standard_tokens:
            if word in prediction_tokens:
                recall += 1
        recall /= len(golden_standard_tokens)
        
        total_recall += recall
    return (total_recall/len(question_set))

def evaluate_answer_precision(question_set, answers):
    total_precision = 0
    for i in range(len(question_set)):
        prediction_tokens = word_tokenize(answers[i])
        golden_standard_tokens = word_tokenize(question_set[i]["text"])
        
        # precision
        precision = 0
        for word in prediction_tokens:
            if word in golden_standard_tokens:
                precision += 1
        if (len(prediction_tokens) == 0):
            precision = 0
        else:
            precision /= len(prediction_tokens)
        
        total_precision += precision
    return (total_precision/len(question_set))

Define a function for writing the answers to the "result.csv" file.

In [12]:
def writeToFile(answers):
    result = []
    for i in range(len(answers)):
        answer_dict = {}
        answer_dict["id"] = i
        answer_dict["answer"] = answers[i]
        result.append(answer_dict)

    with open("result.csv","wb") as myFile:
        fieldnames = ['id', 'answer']
        writer = csv.DictWriter(myFile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(result)
    myFile.close()

## Section 4: Basic Method

Use the same method to find the best sentence index and store in the question_set as a new attribute.

In [13]:
def find_best_sent(question_set):
    for info in question_set:
        # Get query terms
        query_terms = get_BOW(info["question"]).keys()
        
        para = docs[info["docid"]]["text"][info["best_para_id"]]
        sents = sent_tokenize(para)
        
        # Get inverted_index for the paragraph
        inverted_index = get_inverted_index(sents)
        
        # Find the index of the best paragraph
        info["best_sent_id"] = query_vsm(query_terms, inverted_index)

Define a method to obtain the answers by applying this basic method.

In [14]:
def basic_method(question_set):
    # calculate best paragraph and sentences for the question_set
    find_best_paras(question_set)
    find_best_sent(question_set)
    
    # Obtain the answers
    answers = []
    for info in question_set:
        para = docs[info["docid"]]["text"][info["best_para_id"]]
        sents = sent_tokenize(para)
        best_sent = sents[info["best_sent_id"]]
        query_terms = get_BOW(info["question"]).keys()
        
        # find keywords in the best sentence
        answer_set = set()  # without pos tag conditiion
        answer_set_pos = set()  # with pos tag conditiion, more stricted than answer_set_pos
        tagged_sent = pos_tag(word_tokenize(best_sent))  # pos tagged sentence
        
        for (token, tag) in tagged_sent:
            if ((token not in stop_words) and (not re.match("[^\w\s]", token)) # remove stop words and punctuations
                and (token not in query_terms)): # answers usually not in query terms
                answer_set.add(token)
                if (tag in ["NNP", "CD", "NN", "NNS", "NNPS", "FW"]): # leave nouns and numbers and foreign words only (more stricted)
                    answer_set_pos.add(token)
                
        # answer string
        answer_str = ""
        
        if (len(answer_set_pos) != 0):  # use strict one if possible
            for word in list(answer_set_pos):
                answer_str += word + " "
        else:
            for word in list(answer_set):
                answer_str += word + " "
            
        answers.append(answer_str)
            
    return answers

Apply this basic method to the development and training set set and evaluate for error analysis

In [41]:
combined_answers = basic_method(devel_set + training_set)
print("basic fscore: " + str(evaluate_answer(devel_set + training_set, combined_answers)))

basic fscore: 0.09136886623952355


Calculate the average recall and recision for the development set answers.

In [42]:
print("basic average recall: " + str(evaluate_answer_recall(devel_set + training_set, combined_answers)))
print("basic average precision: " + str(evaluate_answer_precision(devel_set + training_set, combined_answers)))

basic average recall: 0.2570719039832153
basic average precision: 0.06325076121638475


Apply this basic method to the test set for Kaggle submission. (Score: 0.18149)

In [19]:
test_answers = basic_method(test_set)
writeToFile(test_answers)  # write updated answers to file

## Section 5: Advanced Method

Label the question type of each question and subtype if possible. (what, who, where, how, which, when)

In [23]:
def label_type(question_set):
    # question keywords
    types = ["what", "who", "where", "whose", "whom", "how", "which", "when"]
    for info in question_set:
        
        tokenized_question = word_tokenize(info["question"])
        tokenized_question = [word.lower() for word in tokenized_question]  # all tokens to lower case
        info["type"] = "unknown"  # init as unknown
        
        # labelling question type
        for token in tokenized_question:
            if token in types:
                info["type"] = token
                break
                
        # labelling subtype for what, which, and how questions
        if(info["type"] == "what" or info["type"] == "which" or info["type"] == "how"):
            info["subtype"] = find_subtype(info["type"], tokenized_question)
                
        if (info["type"] == "whose" or info["type"] == "whom"):  # Converting "whose" and "whom" to "who"
            info["type"] = "who"
            
        elif (info["type"] == "unknown" and ("or" in tokenized_question)):  # finding "or" questions
            info["type"] = "or"

In [24]:
# first noun in query_terms as subtype name
def find_subtype(question_type, tokenized_question):
     
    # find first noun in query_terms
    subtype = "unknown"
    
    index = tokenized_question.index(question_type)  # index of "what", "which", or "how"
    part_tokens = tokenized_question[index:]  # only take words after the "what", "which", or "how"
    
    if question_type == "how":  # how: (much) or (number)
        num_ls = ["many", "tall", "old", "long", "large"]
        for element in num_ls:
            if element == "much":
                subtype = "much"   # can be money
            if element in part_tokens:
                subtype = "number"
                break
    else:  # what and which
        for (token, tag) in pos_tag(part_tokens):
            if(tag in ["NNP", "NN", "NNS", "NNPS"]):  # noun
                subtype = lemmatizer.lemmatize(token, "n")
                break
    
    return subtype

Label the whole training_set.

In [25]:
label_type(training_set)

Find the entities for the "what" and "which" questions in the training set.

In [26]:
what_X = []
what_Y = []
which_X = []
which_Y = []

In [27]:
for info in training_set:
    if (info["type"] == "what" or info["type"] == "which"):
        ne_tags = nlp(info["text"]).ents
        if (ne_tags):  # can label, then add for training
            if (info["type"] == "what"):  # what
                what_X.append(info["subtype"])
                what_Y.append(ne_tags[0].label_)
            else:  # which
                which_X.append(info["subtype"])
                which_Y.append(ne_tags[0].label_)

Convert into X and Y numbers for machine learning.

In [28]:
answer_type_ls = ["PERSON","NORP","FAC","ORG","GPE","LOC","PRODUCT","EVENT",
                  "WORK_OF_ART","LAW","LANGUAGE","DATE","TIME","PERCENT","MONEY",
                  "QUANTITY","CARDINAL", "ORDINAL"] # all possible values of Y    

In [29]:
# all possible subsets for what and which questions
what_X_vocab = set()
which_X_vocab = set()
for x in what_X:
    what_X_vocab.add(x)
for x in which_X:
    which_X_vocab.add(x)
    
# convert to list
what_X_vocab = list(what_X_vocab)
which_X_vocab = list(which_X_vocab)

print(len(what_X_vocab))
print(len(which_X_vocab))

635
129


In [30]:
# new training data with numbers (index) only
new_what_X = []
new_what_Y = []
new_which_X = []
new_which_Y = []

In [31]:
for x in what_X:
    one_hot = []
    for i in range(len(what_X_vocab)):
        if (i == what_X_vocab.index(x)):
            one_hot.append(1)
        else:
            one_hot.append(0)
    new_what_X.append(one_hot)
for y in what_Y:
    new_what_Y.append(answer_type_ls.index(y))
for x in which_X:
    one_hot = []
    for i in range(len(which_X_vocab)):
        if (i == which_X_vocab.index(x)):
            one_hot.append(1)
        else:
            one_hot.append(0)
    new_which_X.append(one_hot)
for y in which_Y:
    new_which_Y.append(answer_type_ls.index(y))

Check the length of the training data.

In [32]:
print(len(new_what_X))
print(len(new_what_Y))
print(len(new_which_X))
print(len(new_which_Y))

4813
4813
443
443


Training linear GaussianNB classifiers for what and which.

In [33]:
gnb_what = GaussianNB().fit(new_what_X, new_what_Y)
gnb_which = GaussianNB().fit(new_which_X, new_which_Y)

Extract X and Y from development data for testing.

In [34]:
label_type(devel_set)

# What
X_develop_what = []
true_y_develop_what = []

# Which
X_develop_which = []
true_y_develop_which = []

for info in devel_set:
    
    question_type = info["type"]
    
    if (question_type == "what"):
        # predict answer type
        subtype = info["subtype"]

        if subtype in what_X_vocab:
            
            # NE tagging
            ne_tags = nlp(info["text"]).ents
            if (ne_tags):  # can label, then add for training

                # Getting true Y
                true_y_develop_what.append([answer_type_ls.index(ne_tags[0].label_)])

                # Getting X
                one_hot = []
                for i in range(len(what_X_vocab)):
                    if (i == what_X_vocab.index(x)):
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                X_develop_what.append(one_hot)
    elif (question_type == "which"):
        # predict answer type
        subtype = info["subtype"]

        if subtype in which_X_vocab:
            # NE tagging
            ne_tags = nlp(info["text"]).ents
            if (ne_tags):  # can label, then add for training

                # Getting true Y
                true_y_develop_which.append([answer_type_ls.index(ne_tags[0].label_)])

                # Getting X
                one_hot = []
                for i in range(len(which_X_vocab)):
                    if (i == which_X_vocab.index(x)):
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                X_develop_which.append(one_hot)

Training DecisionTreeClassifier for what and which.

In [35]:
dtc_what = DecisionTreeClassifier().fit(new_what_X, new_what_Y)
dtc_which = DecisionTreeClassifier().fit(new_which_X, new_which_Y)

Decision tree has better performance comparing with NB

In [36]:
print("NB what score: " + str(gnb_what.score(X_develop_what, true_y_develop_what)))
print("NB which score: " + str(gnb_which.score(X_develop_which, true_y_develop_which)))
print("DT what score: " + str(dtc_what.score(X_develop_what, true_y_develop_what)))
print("DT which score: " + str(dtc_which.score(X_develop_which, true_y_develop_which)))

NB what score: 0.0168067226891
NB which score: 0.611111111111
DT what score: 0.456582633053
DT which score: 0.611111111111


Create a dict to map each subtype in what and which to a NE tag.

In [37]:
what_subtype_tag_dict = {}
for i in range(len(what_X_vocab)):
    one_hot = []
    for j in range(len(what_X_vocab)):
        if (j == i):
            one_hot.append(1)
        else:
            one_hot.append(0)
    predict_y = dtc_what.predict([one_hot])[0]
    what_subtype_tag_dict[what_X_vocab[i]] = predict_y

which_subtype_tag_dict = {}
for i in range(len(which_X_vocab)):
    one_hot = []
    for j in range(len(which_X_vocab)):
        if (j == i):
            one_hot.append(1)
        else:
            one_hot.append(0)
    predict_y = dtc_which.predict([one_hot])[0]
    which_subtype_tag_dict[which_X_vocab[i]] = predict_y

Define a function to filter the answer string according to the NE tags.

In [38]:
def filter_answer(answer_str, tags):
    ents = nlp(answer_str).ents
    if (ents):  # can label
        filtered_tokens = []
        for ent in ents:
            if ent.label_ in tags:
                filtered_tokens.append(ent.text)
        if (len(filtered_tokens) != 0):  # target NE found
            new_str = ""
            for token in filtered_tokens:
                new_str += token + " "
            return new_str
        else:  # use original answer
            return answer_str
    else:  # cannot label, use original answer
        return answer_str

Dictionary that maps the simple question types to expected answer types.

In [39]:
question_ne_dict = {"who": ["PERSON"], "where": ["GPE", "LOC", "FAC"], "when": ["DATE", "TIME"]}
how_ne_dict = {"much": ["QUANTITY", "CARDINAL", "MONEY"], "number": ["QUANTITY", "CARDINAL", "PERCENT"]}

Define the advanced method.

In [40]:
def advanced_method(question_set):
    
    answers = basic_method(question_set)  # apply basic method first
    label_type(question_set)  # label question types
    
    for i in range(len(question_set)):
        info = question_set[i]
        question_type = info["type"]
        answer = answers[i]  # answers from basic method
        
        if question_type in ["who", "where", "when"]:  # who, where, when questions
            answers[i] = filter_answer(answer, question_ne_dict[question_type])
            
        elif question_type == "how":   # how questions
            subtype = info["subtype"]
            if subtype in how_ne_dict.keys():
                answers[i] = filter_answer(answer, how_ne_dict[subtype])
                
        elif question_type == "or":  # or questions
            # Find the best sentence
            query_terms = word_tokenize(info["question"])
            
            # use the words before and after "or" as answer
            index = query_terms.index("or")
            if (index != 0 and index != len(query_terms) - 2):
                before = query_terms[index -1]
                after = query_terms[index + 1]
                answers[i] = before + " " + after
        
        elif question_type == "what":  # what questions
            # predict answer type
            subtype = info["subtype"]
            if subtype in what_X_vocab:
                answer_tag = answer_type_ls[what_subtype_tag_dict[subtype]]
                answers[i] = filter_answer(answer, [answer_tag])
                
        elif question_type == "which":  # which questions
            # predict answer type   
            subtype = info["subtype"]
            if subtype in which_X_vocab:
                answer_tag = answer_type_ls[which_subtype_tag_dict[subtype]]
                answers[i] = filter_answer(answer, [answer_tag])
    return answers

Apply this basic method to the development set and evaluate.

In [43]:
devel_answers_basic = basic_method(devel_set)
print("basic fscore: " + str(evaluate_answer(devel_set, devel_answers_basic)))

basic fscore: 0.08337461679547321


Apply the basic method to the development set and evaluate.

In [45]:
devel_answers_advanced = advanced_method(devel_set)
print("advanced fscore: " + str(evaluate_answer(devel_set, devel_answers_advanced)))

advanced fscore: 0.11968652329882037


Apply the advanced method to the development set and evaluate.

In [46]:
print("basic average recall: " + str(evaluate_answer_recall(devel_set, devel_answers_basic)))
print("basic average precision: " + str(evaluate_answer_precision(devel_set, devel_answers_basic)))
print("advanced average recall: " + str(evaluate_answer_recall(devel_set, devel_answers_advanced)))
print("advanced average precision: " + str(evaluate_answer_precision(devel_set, devel_answers_advanced)))

basic average recall: 0.2420514476374988
basic average precision: 0.05696210319977588
advanced average recall: 0.22240878269292874
advanced average precision: 0.10196751196946854


Apply this advanced method to the test set for Kaggle submission. (Score: 0.24050)

In [47]:
test_answers = advanced_method(test_set)
writeToFile(test_answers)  # write updated answers to file